In [ ]:
from IPython import get_ipython
my_variables = set(dir())  # Write this line at the beginning of cell
my_variables = list(set(dir()) - my_variables)  # Write these 2 lines at the end of cell
get_ipython().magic('%reset_selective -f [{}]'.format(','.join(my_variables)))

import cv2 as cv
import numpy as np
import numpy.ma as ma
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw 
import pylab

In [ ]:
def plt_show(image, color = None):
    pic_size=5
    plt.figure(figsize = (pic_size, pic_size))
    plt.imshow(image, color)
    plt.show()

In [ ]:
def otsuFragment(pix, old):
    hist = hist_ar(pix)

    summ = 0
    for x in range(len(pix)):
        summ = summ +pix[x]

    indexMax = 0
    sigMax = 0.0 

    l1 = 0 
    o1 = 0
    
    if len(old)==0:
        old_new=[0]

    for thresh in range(1,len(hist)): 
       
        if not thresh in old:
            l1 = l1 +  hist[thresh]
            
            # print("o1 ",o1)
            o1 =o1+ thresh * hist[thresh]
    
            # print("l1",l1)
            omega1 = l1 /len(pix) 
            omega2 = 1.0 - omega1 
    
            lyamda1 = o1 /l1
            lyamda2 = (summ - o1)/(len(pix) - l1)
            
            between = lyamda1 - lyamda2
    
            sigma = omega1 * omega2 * between * between

            if (sigma > sigMax): 
                sigMax = sigma
                indexMax = thresh
    return indexMax 

def hist_ar(pixarr):
        h = np.zeros(256, np.float32)
        for x in range(len(pixarr)):
            h[round(pixarr[x])]=h[round(pixarr[x])]+1
        for i in range(len(h)):
            if len(h)>i:
                if ((h[i]==0) & (len(h)>0)):
                    h=np.setdiff1d(h, 0)
                    i = i - 1
        return h
    
def otsu_loc(image_name):
    partsH = 2;  partsV = 2
    
    ref = cv.imread(image_name, cv.IMREAD_GRAYSCALE)
    image = Image.open(image_name)
    new_pix = ImageDraw.Draw(image)  
    
    sizeV = image.size[1];  sizeH = image.size[0]

    stepV = np.array([x for x in range(0,sizeV-sizeV//partsH+1,int(sizeV//partsH))])  
    stepV=np.append(stepV,sizeV)
    
    stepH = np.array([x for x in range(0,sizeH-sizeH//partsV+1,int(sizeH//partsV))])    
    stepH=np.append(stepH, sizeH)

    pixarr = cv.imread(image_name, cv.IMREAD_GRAYSCALE)
    for i in range(len(stepV)-1):
        for j in range(len(stepH)-1):
            arr1 = arr = (pixarr[stepV[i]:stepV[i+1],stepH[j]:stepH[j+1]])
            limit = otsuFragment(arr1.reshape(len(arr)*len(arr[0])), [])
            for x in range(stepV[i],stepV[i+1]):
                for y in range(stepH[j],stepH[j+1]):
                    if arr[x-stepV[i],y-stepH[j]] < limit:
                        pix = 0
                    else:
                        pix = 255
                    ref[x][y]=pix
                    new_pix.point((y, x), (pix, pix, pix)) 
    fig, ax = plt.subplots(1,2)
    plt.title("Local binarization Otsu's method")
    ax[0].imshow( Image.open(image_name) )
    ax[1].imshow(image)
    plt.show()
    return ref

In [ ]:
bin_ref = otsu_loc("pic2.jpg")

In [ ]:
circle = cv.getStructuringElement(cv.MORPH_ELLIPSE, (15, 18))# np.ones((9,2),np.uint8)
spoons_sugar = cv.erode(bin_ref, circle)
plt_show(spoons_sugar)

spoons_sugar = cv.dilate(spoons_sugar, circle)
plt_show(spoons_sugar)

In [ ]:
contours, _ = cv.findContours(spoons_sugar,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
contour=[]
for i in contours:
    if(len(i)<180):
        contour.append(i)
print("Count of sugar = ", len(contour))
figures_border = cv.drawContours(cv.imread("pic2.jpg"),contour,-1,(14,28,57), thickness=10)
plt_show(figures_border)

# CV_CHAIN_APPROX_SIMPLE — склеивает все горизонтальные, вертикальные и диагональные контуры.


In [ ]:
contour=[]
for i in contours:
    if(len(i) > 180):
        contour.append(i)
print("Count of spoons = ", len(contour))
figures_border = cv.drawContours(cv.imread("pic2.jpg"),contour,-1,(14,28,57), thickness=10)
plt_show(figures_border)

In [ ]:
for i in contours:
    if(len(i)<180):
        contour.append(i)
print("Total count = ", len(contours))
figures_border = cv.drawContours(cv.imread("pic2.jpg"),contours,-1,(14,28,57), thickness=10)
plt_show(figures_border)